# Emotional Analizer
trainig pretrained model to analize emotions in text using fine-tuning

**Warinig** : The following code executes only on Google Colab

In [ ]:
!pip install transformers
!pip install nlp

## Load the model and tokenizer

In [1]:
from transformers import BertTokenizerFast, BertForSequenceClassification
import torch
import nlp

# Load the BERT model
sc_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

2023-03-29 18:46:23.083296: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertFor

## Load the dataset

In [2]:
# load the IMDb dataset from nlp
from nlp import load_dataset
train_data, test_data = load_dataset('imdb', split=['train', 'test'])

# define the batch tokenizer
def batch_tokenizer(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

# tokenize the data
train_data = train_data.map(batch_tokenizer, batched=True, batch_size=len(train_data))
test_data = test_data.map(batch_tokenizer, batched=True, batch_size=len(test_data))

# format the data
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Downloading:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

KeyboardInterrupt: 

In [ ]:
# define the function to evaluate
from skearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}